In [1]:
import rebound
import time
import sys
import numpy as np
from matplotlib import pyplot as plt
from astropy import units as u
from astropy import constants as c

# local imports
import heartbeat
import globs
import metasim

In [2]:
def run_sim(filestem='test/test'):
    
    # global variables

    globs.initialise()

    globs.glob_dclo = 1. #CE distance to check in rH
    #dir = 'test/'
    globs.glob_archive = ''
    #globals.glob_names = []
    globs.glob_is_close = False

    sys = metasim.MetaSim()
    sys.run_planets()
    if globs.glob_is_close:
        sys.rewind()
        sys.add_moons()
        sys.run_moons()
        sys.analyse()

In [3]:
Nsys = 1

for i in range(Nsys):
    run_sim(filestem=f'test/test{i:04d}')

Simulation test/test0000.

Creating new simulation...
Running planets-only simulation...
CE between Planet1 and Planet2 with dist 0.014383669130648079 au at 366.9122590222095 years
<rebound.particle.Particle object at 0x7fadf0ccecc0, m=0.0019091884679386499 x=-1.7375181783728353 y=0.7815401438258468 z=0.00576193228393899 vx=0.5341230289757545 vy=-4.447504153092214 vz=0.8912802046243457>
<rebound.particle.Particle object at 0x7fadf0ccef40, m=0.0019091884679386499 x=-1.7340496823546139 y=0.7953745803816724 z=0.007624145540893367 vx=-3.3621243698742647 vy=-3.1072381846854142 vz=-0.7996263471465357>
CE between Planet2 and Planet1 with dist 0.014383669130648079 au at 366.9122590222095 years
<rebound.particle.Particle object at 0x7fadf0ccec40, m=0.0019091884679386499 x=-1.7340496823546139 y=0.7953745803816724 z=0.007624145540893367 vx=-3.3621243698742647 vy=-3.1072381846854142 vz=-0.7996263471465357>
<rebound.particle.Particle object at 0x7fadf0ccebc0, m=0.0019091884679386499 x=-1.7375181783

/Users/alexander/research/circumplanetary_discs/rebound/rebound/rebound/simulation.py:712: RuntimeWarning: File in use for SimulationArchive already exists. Snapshots will be appended.
  warnings.warn(msg[1:], RuntimeWarning)


CE between Planet1 and Planet2 with dist 0.014489848514046104 au at 366.9127390697916 years
<rebound.particle.Particle object at 0x7fae404b09c0, m=0.0019091884679386499 x=-1.7386084024369375 y=0.7799507578584187 z=0.005617658205855432 vx=0.5128937088675674 vy=-4.509964662895584 vz=0.8758281111364068>
<rebound.particle.Particle object at 0x7fae404b0cc0, m=0.0019091884679386499 x=-1.7343853063490544 y=0.7936372154045849 z=0.007809016056481928 vx=-3.333492636659222 vy=-3.0478810257018862 vz=-0.7841327464797062>
CE between Planet2 and Planet1 with dist 0.014489848514046104 au at 366.9127390697916 years
<rebound.particle.Particle object at 0x7fae404b0d40, m=0.0019091884679386499 x=-1.7343853063490544 y=0.7936372154045849 z=0.007809016056481928 vx=-3.333492636659222 vy=-3.0478810257018862 vz=-0.7841327464797062>
<rebound.particle.Particle object at 0x7fae404b08c0, m=0.0019091884679386499 x=-1.7386084024369375 y=0.7799507578584187 z=0.005617658205855432 vx=0.5128937088675674 vy=-4.50996466289

Moons simulation complete
1000.0 years took 24.19414973258972 seconds


/Users/alexander/research/circumplanetary_discs/rebound/rebound/rebound/simulationarchive.py:146: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


NameError: name 'name_pl' is not defined

In [ ]:
  
def setup_sim_moon(i):

    global glob_archive
    global glob_is_close
    global glob_is_eject
    global glob_is_stop
    global glob_dclo
    global glob_pl_done
    
    print()
    print(f'Simulation {i} with moons:')
    
    globals.glob_archive = dir + f"moons_{i:04d}.bin"
    globals.glob_log = globals.glob_archive[:-3]+'.log'
    globals.glob_is_close = False
    globals.glob_is_eject = False
    globals.glob_is_stop = False
    
    pl_done = (globals.glob_archive in globals.glob_pl_done)

    try:
        sim = rebound.Simulation(globals.glob_archive)
    except:
        print(f'archive file {gloabls.glob_archive} not found')
        return
    else:
#check end state
        if len(sim.particles) == 2:
            print('restored simulation has only one planet')
            print()
            glob_is_eject = True
        if sim.t >= tmax - TINY:
            print(f'restored simulation was stable for {tmax} yrs')
            print()
            glob_is_stop = True
        for p in sim.particles[1:]:
            rh2 = p.d**2 * (p.m/(3*sim.particles[0].m))**(2/3)
            for q in sim.particles[1:]:
                if p.hash.value == q.hash.value:
                    continue
                dx = p.x-q.x
                dy = p.y-q.y
                dz = p.z-q.z
                d2 = dx*dx + dy*dy + dz*dz
                if d2 <= rh2*globals.glob_dclo:
                    globals.glob_is_close = True
                    print('restored simulation is undergoing CE')
                    print()
        if ((not globals.glob_is_eject) and (not globals.glob_is_stop) and 
            (not globals.glob_is_close) and (not pl_done)):
            print('restored simulation needs continuing...')
            run_sim_pl(i)

# if CE, rewind and add moons

        rng = np.random.default_rng()
        
        has_moons = archive in globals.glob_moons_added.keys()
        
        if globals.glob_is_close and not has_moons:
            
            print('rewinding simulation...')
            sim.dt = -sim.dt
            tstart = sim.t - t_rewind
            sim.integrate(tstart)
            
            print('adding moons...')
            sim.dt = -sim.dt
            
            for p in sim.particles[1:]:
                p.r = (((9/4*np.pi)*p.m*u.Msun/(1.8*u.g/u.cm**3))**(1/3)).to_value(u.au)
                print(p.r)
            # moons. Data from JPL horizons 2022-01-26
            Nmoonsppl = 4 #per planet
            amo = (np.array([4.220279893619238E+05,6.712942692154385E+05,1.070941243876099E+06,
                             1.883987339695996E+06]) * u.km).to(u.au)
            emo = np.array([3.779947596328282E-03,9.515599858678114E-03,1.437934948643515E-03,
                            7.429672504931133E-03])
            imo = np.radians([2.233261880507145E+00,2.499911845043659E+00,2.320207222240029E+00,
                              1.964066225350317E+00])
            wmo = np.radians([2.230114025429823E+02,5.871258018281303E+01,3.520404542215327E+02,
                              3.411426411826523E+01]) # arg of peri
            Omo = np.radians([3.367638701787085E+02,3.289828238020597E+02,3.401167587299205E+02,
                              3.370179889677478E+02])
            mamo = np.radians([2.411569358232406E+02,1.160555854430442E+02,1.129206855751383E+02,
                               6.172934365391181E+01])
            name_mo_stem = ['Io','Europa','Ganymede','Callisto']
            Rmo = (np.array([1821.49,1560.8,2631.2,2410.3])*u.km).to(u.au)
            Mmo = (np.array([5959.9155,3202.7121,9887.8328,
                             7179.2834])*u.km**3/u.s**2 / c.G).to(u.Msun) #JPL gives product GM

            Omo_offset = np.radians(rng.uniform(0,360,2)) #we rotate Omega for all moons in the same moon system by the same amount
            name_moons = [[m+str(i+1) for m in name_mo_stem] for i in range(Npl)]

            for i in range(Npl):
                for j in range(Nmoonsppl):
                    sim.add(a=amo[j].to_value(u.au),e=emo[j],inc=imo[j],omega=wmo[j],Omega=Omo[j]+Omo_offset[i],
                            M=mamo[j],m=Mmo[j].to_value(u.Msun),r=Rmo[j].to_value(u.au),
                            primary=sim.particles[name_pl[i]],hash=name_moons[i][j])

            sim.move_to_com()
            
            # save a snapshot
            sim.simulationarchive_snapshot(globals.glob_archive)
            
            with open('moons_added.txt','a') as f:
                f.write(f'{globals.glob_archive} {sim.t}\n')
            
        else:
            print('No close encounter: passing to next simulation')
            return
        
#        # automate archive
#        sim.automateSimulationArchive(glob_archive, interval=1.,deletefile=False)
#        
#        tend = glob_moons_added[glob_archive] + tmoons
#        print('integrating with moons...')
#        clock_t0 = time.time()
#        sim_t0 = sim.t
#        sim.integrate(tend)
#        sim_t1 = sim.t
#        clock_t1 = time.time()
#        print(f'{sim_t1-sim_t0} years took {clock_t1-clock_t0} seconds')
#        
#        # final snapshot
#        sim.simulationarchive_snapshot(glob_archive)
            
        return

In [ ]:
# can we restore an existing sim?
try:
    sim = rebound.Simulation()
except:
    

globals.glob_pl_done = []
globals.glob_moons_added = {}

try:
    with open('planets_done.txt','r') as f:
        lines = f.readlines()
except:
    print('No sims with pre-moon run completed')
else:
    for l in lines:
        globals.glob_pl_done.append(l.strip())

try:
    with open('moons_added.txt','r') as f:
        lines = f.readlines()
except:
    print('No simulations with moons yet added')
else:    
    for l in lines:
        s = l.split()
        globals.glob_moons_added[s[0]] = float(s[1])


for i in range(Nsys):
    archive = dir + f"moons_{i:04d}.bin"

# if not in list of sims with planet bit finished, must continue 
    if not (archive in globals.glob_pl_done):
        run_sim_pl(i)

    setup_sim_moon(i)

In [ ]:
try:
    with open('moons_added.txt','r') as f:
        lines = f.readlines()

    for l in lines:
        s = l.split()
        glob_moons_added[s[0]] = float(s[1])
except:
    print('no simulations with moons yet added')
else:
    for s in glob_moons_added.keys():

#        if s == 'test/moons_0000.bin':
#            continue # this one has ground to a halt
        
        glob_log = s[:-3]+'log'
        
        glob_darr = [[[9999.9,9999.9,9999.9],[9999.9,9999.9,9999.9]],
                     [[9999.9,9999.9,9999.9],[9999.9,9999.9,9999.9]]]
        
        archive = s
        sim = rebound.Simulation(s)
        glob_planets = [n for n in name_pl]
        sim_hashes = [p.hash.value for p in sim.particles]
        glob_npl = sum([rebound.hash(p).value in sim_hashes for p in glob_planets])# this and the heartbeat needs to be done more carefully in case planets change their order

        tend = glob_moons_added[s] + t_moons
        sim.heartbeat = heartbeat
        sim.collision = "line"
        sim.collision_resolve = logged_merge
        sim.track_energy_offset = 0

        sim.automateSimulationArchive(s,interval=1.,deletefile=False)
        
        sim_t0 = sim.t
        clock_t0 = time.time()

        print(f'Simulation restored at {sim.t} years:    {s}')
        print()
        
        with open(glob_log,'a') as f:
            print(f'Simulation restored at {sim.t} years:    {s}',file=f)
            print('',file=f)

        
        while sim.t < tend:
            try:
                sim.integrate(tend)
            except rebound.Escape as error:
# save at this point
                sim.simulationarchive_snapshot(archive)

                print(error)
                hashes = set()
# Rebound example just allows one body to be removed; might have more    
                for h in name_all:
                    try:
                        p = sim.particles[h]
                        d2 = p.x**2 + p.y**2 + p.z**2
                        if d2 > sim.exit_max_distance**2:
                            hashes.add(h)
                    except:
                        if verbose:
                            print(f'{h} has already been removed')

# Here, we also want to remove any moons bound to a removed planet
                for h in name_all:
                    try:
                        if rebound.hash(find_primary(h,name_pl,sim)) in [ha.value for ha in hashes]:
                            hashes.add(h)
                    except:
                        if verbose:
                            print(f'{h} has already been removed')


                Ein = sim.calculate_energy()

                for h in hashes:
                    print(f'{h} ejected at {sim.t} years')
                    print(sim.particles[h])
                    with open(globals.glob_log,'a') as f:
                        print(f'{h} ejected at {sim.t} years',file=f)
                        print(sim.particles[h],file=f)
                    if h in globals.glob_planets:
                        globals.glob_planets.remove(h)
                        globals.glob_npl = globals.glob_npl-1
                    sim.remove(hash=h)
                sim.move_to_com()    

                Eout = sim.calculate_energy()

#        if sim.track_energy_offset:
                sim.energy_offset += (Ein-Eout)

        sim_t1 = sim.t
        clock_t1 = time.time()
        
        print(f'{sim_t1-sim_t0} years took {clock_t1-clock_t0} seconds')
        print()
        
        with open(glob_log,'a') as f:
            print(f'{sim_t1-sim_t0} years took {clock_t1-clock_t0} seconds',file=f)
            print('',file=f)
        
        for moon in name_all:
            try:
                print(moon+' bound to '+find_primary(moon,name_pl,sim).hash)
                with open(glob_log,'a') as f:
                    print(moon+' bound to '+find_primary(moon,name_pl,sim).hash,file=f)
            except AttributeError:
                print(moon+' was removed')
                with open(glob_log,'a') as f:
                    print(moon+' was removed',file=f)

        print()
        print()
        
        with open(glob_log,'a') as f:
            print('',file=f)
            print('',file=f)